### Laboratorio de Escalonamento ###

Neste laboratório, iremos simular o funcionamento de algoritmos de escalonamento básicos para entender melhor seu funcionamento.

Na célula abaixo, temos uma classe Processo, que tem as informações de execução, e uma classe de exemplo de escalonamento apenas com os protótipos:

(orientação a objeto em Python)

In [152]:
import numpy as np
import random
import sys
sys.setrecursionlimit(10000)

class Processo(object):
    def __init__(self, pnome, pio, ptam, prioridade, tempoChegada):
        self.nome = pnome
        self.io = pio  
        self.tam = ptam  
        self.prio = prioridade  
        self.chegada = tempoChegada  
        self.tempo_restante = ptam  
        self.tempo_inicio = None  
        self.tempo_final = None  

    def roda(self, tempo, quantum=None):  
        if self.tempo_inicio is None:
            self.tempo_inicio = tempo 

        if random.randint(1, 100) < self.io:  
            self.tempo_restante -= 1
            print(self.nome, "realizou I/O, tempo restante", self.tempo_restante)
            return 1, True  

        if quantum is None or self.tempo_restante < quantum:
            quantum = self.tempo_restante
        self.tempo_restante -= quantum

        if self.tempo_restante == 0:
            self.tempo_final = tempo + quantum  

        print(self.nome, "Rodou por", quantum, "timeslice(s), tempo restante", self.tempo_restante)
        return quantum, False  

class escalonador(object):  
    def __init__(self, vprontos=[]):
        self.prontos = vprontos.copy()  

    def pronto(self, processo):
        self.prontos.append(processo)

    def proximo(self):
        if self.prontos:
            return self.prontos.pop(0)  
        else:
            return None



Na célula abaixo, são criados quatro processos completamente CPU-Bound que precisam de 3 timeslices para rodar.

O valor de E/S é um número entre 0 e 100 indicando quantos porcento de chance o processo tem de fazer E/S durante seu tempo na CPU

In [153]:
nprocs = 4
nomes = ['A', 'B', 'C', 'D']
chanceio = [0, 0, 0, 0]  
tamanho = np.array([3, 3, 3, 3])

total = tamanho.sum()
procs = []

for i in range(nprocs):
    procs.append(Processo(nomes[i], chanceio[i], tamanho[i], 0, 0))  

Na célula abaixo, temos uma simulação do funcionamento de um escalonador de processos. As duas configurações importantes aqui são o valor do quantum padrão (que pode ser dinamico em algoritmos mais complexos, e quantos timeslices um processo que faz e/s passa bloqueado.

Percebam que na terceira linha é instanciado o escalonador (neste caso, um round_robin). Isto foi feito assim para ser simples trocar o escalonador e repetir a simulação, bastando criar uma classe com os métodos pronto, proximo e construtor e alterar esta linha.

In [154]:
quantum = 2
tempoBloq = 1
escalonador = escalonador(procs)  
bloqueados = []
tempo = 0  
random.seed(0)

# Simulation loop
while total > 0:
    p = escalonador.proximo()
    if p is not None:
        
        tempo_atual = tempo

        if p.tempo_restante < quantum:
            quantum_atual = p.tempo_restante
        else:
            quantum_atual = quantum

        
        rodou, _ = p.roda(tempo_atual, quantum=quantum)
        tempo += rodou  
        total -= rodou

        if p.tempo_restante > 0:
            escalonador.pronto(p)
    else:
        
        tempo += 1


tempo_de_execucao = []
tempo_de_resposta = []

for proc in procs:
    if proc.tempo_final is not None and proc.tempo_inicio is not None:
        tempo_execucao = proc.tempo_final - proc.chegada
        tempo_resposta = proc.tempo_inicio - proc.chegada
        tempo_de_execucao.append(tempo_execucao)
        tempo_de_resposta.append(tempo_resposta)
        print(f"Processo {proc.nome}: Tempo de Execucao = {tempo_execucao}, Tempo Resposta = {tempo_resposta}")
    else:
        print(f"Processo {proc.nome}: Não terminou a execução.")

tempo_medio_execucao = sum(tempo_de_execucao) / len(tempo_de_execucao)
tempo_medio_resposta = sum(tempo_de_resposta) / len(tempo_de_resposta)

print(f"\nTempo Medio de Execucao: {tempo_medio_execucao}")
print(f"Tempo Medio de Resposta: {tempo_medio_resposta}")

    

A Rodou por 2 timeslice(s), tempo restante 1
B Rodou por 2 timeslice(s), tempo restante 1
C Rodou por 2 timeslice(s), tempo restante 1
D Rodou por 2 timeslice(s), tempo restante 1
A Rodou por 1 timeslice(s), tempo restante 0
B Rodou por 1 timeslice(s), tempo restante 0
C Rodou por 1 timeslice(s), tempo restante 0
D Rodou por 1 timeslice(s), tempo restante 0
Processo A: Tempo de Execucao = 9, Tempo Resposta = 0
Processo B: Tempo de Execucao = 10, Tempo Resposta = 2
Processo C: Tempo de Execucao = 11, Tempo Resposta = 4
Processo D: Tempo de Execucao = 12, Tempo Resposta = 6

Tempo Medio de Execucao: 10.5
Tempo Medio de Resposta: 3.0


### Laboratório: ###


1 - Altere o simulador acima para calcular o tempo de execucao medio e tempo de resposta 

2 - Implemente o escalonador por FIFO e SJF e verifique seus tempos de execução e espera.

3 - Faça em outra célula uma implementação do STCF e Round Robin, alterando o p.roda() para receber o quantum


Na segunda feira (15) haverá um questionário no ColabWeb onde você deverá utilizar suas implementações para responder às perguntas. Na terça de manhã farei uma breve conferência para tirar dúvidas.

### 2 Implemente o Escalonador FIFO e SJF

In [155]:
import numpy as np
import random
import sys
sys.setrecursionlimit(10000)


class Processo(object):
    def __init__(self, pnome, pio, ptam, prioridade, tempoChegada):
        self.nome = pnome
        self.io = pio  
        self.tam = ptam  
        self.prio = prioridade  
        self.chegada = tempoChegada  
        self.tempo_restante = ptam  
        self.tempo_inicio = None  
        self.tempo_final = None  

    def roda(self, tempo, quantum=None):  
        if self.tempo_inicio is None:
            self.tempo_inicio = tempo  

        if random.randint(1, 100) < self.io:  
            self.tempo_restante -= 1
            print(self.nome, "realizou I/O, tempo restante", self.tempo_restante)
            return 1, True  

        if quantum is None or self.tempo_restante < quantum:
            quantum = self.tempo_restante
        self.tempo_restante -= quantum

        print(self.nome, "rodou por", quantum, "timeslice(s), tempo restante", self.tempo_restante)
        return quantum, False  

class EscalonadorFIFO(object):
    def __init__(self, vprontos=[]):
        self.prontos = vprontos.copy()  

    def pronto(self, processo):
        self.prontos.append(processo)

    def proximo(self):
        if self.prontos:
            return self.prontos.pop(0)  
        else:
            return None

class EscalonadorSJF(object):
    def __init__(self, vprontos=[]):
        self.prontos = vprontos.copy()  

    def pronto(self, processo):
        self.prontos.append(processo)

    def proximo(self):
        if self.prontos:
            
            self.prontos.sort(key=lambda x: x.tempo_restante)
            return self.prontos.pop(0)  
        else:
            return None




In [156]:
nprocs = 4
nomes = ['A', 'B', 'C', 'D']
chanceio = [0, 0, 0, 0]  
tamanho = np.array([6, 3, 1, 7])  
procs = []
for i in range(nprocs):
    procs.append(Processo(nomes[i], chanceio[i], tamanho[i], 0, 0))  

In [157]:
def executar_simulacao(escalonador_classe, quantum=2):
    
    for proc in procs:
        proc.tempo_restante = proc.tam
        proc.tempo_inicio = None
        proc.tempo_final = None

    total = sum(proc.tam for proc in procs)
    escalonador = escalonador_classe(procs)
    tempo = 0
    random.seed(0)

    while total > 0:
        p = escalonador.proximo()
        if p is not None:
            tempo_atual = tempo

            if p.tempo_restante < quantum:
                quantum_atual = p.tempo_restante
            else:
                quantum_atual = quantum

            
            rodou, _ = p.roda(tempo_atual, quantum=quantum_atual)
            tempo += rodou  
            total -= rodou

            if p.tempo_restante == 0 and p.tempo_final is None:
                p.tempo_final = tempo  

            if p.tempo_restante > 0:
                escalonador.pronto(p)
        else:
           
            tempo += 1

    
    tempo_de_execucao = []
    tempo_de_resposta = []

    for proc in procs:
        if proc.tempo_final is not None and proc.tempo_inicio is not None:
            tempo_execucao = proc.tempo_final - proc.chegada
            tempo_resposta = proc.tempo_inicio - proc.chegada
            tempo_de_execucao.append(tempo_execucao)
            tempo_de_resposta.append(tempo_resposta)
            print(f"Processo {proc.nome}: Tempo de Execução = {tempo_execucao}, Tempo de Resposta = {tempo_resposta}")
        else:
            print(f"Processo {proc.nome}: Não terminou a execução.")

    tempo_medio_execucao = sum(tempo_de_execucao) / len(tempo_de_execucao)
    tempo_medio_resposta = sum(tempo_de_resposta) / len(tempo_de_resposta)

    print(f"\nTempo Médio de Execução: {tempo_medio_execucao}")
    print(f"Tempo Médio de Resposta: {tempo_medio_resposta}")
    print("-" * 50)

print("Simulação com Escalonador FIFO:")
executar_simulacao(EscalonadorFIFO, quantum=2)

print("Simulação com Escalonador SJF:")
executar_simulacao(EscalonadorSJF, quantum=2)

Simulação com Escalonador FIFO:
A rodou por 2 timeslice(s), tempo restante 4
B rodou por 2 timeslice(s), tempo restante 1
C rodou por 1 timeslice(s), tempo restante 0
D rodou por 2 timeslice(s), tempo restante 5
A rodou por 2 timeslice(s), tempo restante 2
B rodou por 1 timeslice(s), tempo restante 0
D rodou por 2 timeslice(s), tempo restante 3
A rodou por 2 timeslice(s), tempo restante 0
D rodou por 2 timeslice(s), tempo restante 1
D rodou por 1 timeslice(s), tempo restante 0
Processo A: Tempo de Execução = 14, Tempo de Resposta = 0
Processo B: Tempo de Execução = 10, Tempo de Resposta = 2
Processo C: Tempo de Execução = 5, Tempo de Resposta = 4
Processo D: Tempo de Execução = 17, Tempo de Resposta = 5

Tempo Médio de Execução: 11.5
Tempo Médio de Resposta: 2.75
--------------------------------------------------
Simulação com Escalonador SJF:
C rodou por 1 timeslice(s), tempo restante 0
B rodou por 2 timeslice(s), tempo restante 1
B rodou por 1 timeslice(s), tempo restante 0
A rodou p

### 3 Implementar o STCF Round Robin

In [158]:
import numpy as np
import random
import sys
sys.setrecursionlimit(10000)


class Processo(object):
    def __init__(self, pnome, pio, ptam, prioridade, tempoChegada):
        self.nome = pnome
        self.io = pio  
        self.tam = ptam  
        self.prio = prioridade  
        self.chegada = tempoChegada  
        self.tempo_restante = ptam  
        self.tempo_inicio = None 
        self.tempo_final = None 

    def roda(self, tempo, quantum=None):  
        if self.tempo_inicio is None:
            self.tempo_inicio = tempo  

        if random.randint(1, 100) < self.io:  
            self.tempo_restante -= 1
            print(self.nome, "realizou I/O, tempo restante", self.tempo_restante)
            return 1, True  

        if quantum is None or self.tempo_restante < quantum:
            quantum = self.tempo_restante
        self.tempo_restante -= quantum

        print(self.nome, "rodou por", quantum, "timeslice(s), tempo restante", self.tempo_restante)
        return quantum, False  
class EscalonadorSTCF(object):
    def __init__(self, vprontos=[]):
        self.prontos = vprontos.copy()

    def pronto(self, processo):
        self.prontos.append(processo)

    def proximo(self):
        if self.prontos:
            
            self.prontos.sort(key=lambda x: x.tempo_restante)
            return self.prontos.pop(0)
        else:
            return None

class EscalonadorRoundRobin(object):
    def __init__(self, vprontos=[]):
        self.prontos = vprontos.copy()

    def pronto(self, processo):
        self.prontos.append(processo)

    def proximo(self):
        if self.prontos:
            processo = self.prontos.pop(0)
            return processo
        else:
            return None


In [159]:
nprocs = 4
nomes = ['A', 'B', 'C', 'D']
chanceio = [0, 0, 0, 0]
tamanho = np.array([6, 3, 1, 7])  
procs = []
for i in range(nprocs):
    procs.append(Processo(nomes[i], chanceio[i], tamanho[i], 0, 0))

In [160]:
def executar_simulacao(escalonador_classe, quantum=2):
    # Reiniciar os processos
    for proc in procs:
        proc.tempo_restante = proc.tam
        proc.tempo_inicio = None
        proc.tempo_final = None

    total = sum(proc.tam for proc in procs)
    escalonador = escalonador_classe(procs)
    tempo = 0
    random.seed(0)

    while total > 0:
        p = escalonador.proximo()
        if p is not None:
            tempo_atual = tempo

            if p.tempo_restante < quantum:
                quantum_atual = p.tempo_restante
            else:
                quantum_atual = quantum

            # Processo executa
            rodou, _ = p.roda(tempo_atual, quantum=quantum_atual)
            tempo += rodou  
            total -= rodou

            if p.tempo_restante == 0 and p.tempo_final is None:
                p.tempo_final = tempo  

            if p.tempo_restante > 0:
                escalonador.pronto(p)
        else:
            
            tempo += 1

    
    tempo_de_execucao = []
    tempo_de_resposta = []

    for proc in procs:
        if proc.tempo_final is not None and proc.tempo_inicio is not None:
            tempo_execucao = proc.tempo_final - proc.chegada
            tempo_resposta = proc.tempo_inicio - proc.chegada
            print(f"Processo {proc.nome}: Tempo de Execução = {tempo_execucao}, Tempo de Resposta = {tempo_resposta}")
            tempo_de_execucao.append(tempo_execucao)
            tempo_de_resposta.append(tempo_resposta)
        else:
            print(f"Processo {proc.nome}: Não terminou a execução.")

    tempo_medio_execucao = sum(tempo_de_execucao) / len(tempo_de_execucao)
    tempo_medio_resposta = sum(tempo_de_resposta) / len(tempo_de_resposta)

    print(f"\nTempo Médio de Execução: {tempo_medio_execucao}")
    print(f"Tempo Médio de Resposta: {tempo_medio_resposta}")
    print("-" * 50)




print("Simulação com Escalonador STCF:")
executar_simulacao(EscalonadorSTCF, quantum=1)


print("Simulação com Escalonador Round Robin:")
executar_simulacao(EscalonadorRoundRobin, quantum=2) 

Simulação com Escalonador STCF:
C rodou por 1 timeslice(s), tempo restante 0
B rodou por 1 timeslice(s), tempo restante 2
B rodou por 1 timeslice(s), tempo restante 1
B rodou por 1 timeslice(s), tempo restante 0
A rodou por 1 timeslice(s), tempo restante 5
A rodou por 1 timeslice(s), tempo restante 4
A rodou por 1 timeslice(s), tempo restante 3
A rodou por 1 timeslice(s), tempo restante 2
A rodou por 1 timeslice(s), tempo restante 1
A rodou por 1 timeslice(s), tempo restante 0
D rodou por 1 timeslice(s), tempo restante 6
D rodou por 1 timeslice(s), tempo restante 5
D rodou por 1 timeslice(s), tempo restante 4
D rodou por 1 timeslice(s), tempo restante 3
D rodou por 1 timeslice(s), tempo restante 2
D rodou por 1 timeslice(s), tempo restante 1
D rodou por 1 timeslice(s), tempo restante 0
Processo A: Tempo de Execução = 10, Tempo de Resposta = 4
Processo B: Tempo de Execução = 4, Tempo de Resposta = 1
Processo C: Tempo de Execução = 1, Tempo de Resposta = 0
Processo D: Tempo de Execução =